In [ ]:
def bfs(graph, start):
    visited = []
    queue = [start]

    while queue:
        node = queue.pop(0)
        if node not in visited:
            visited.append(node)
            neighbours = graph[node]
            for neighbour in neighbours:
                queue.append(neighbour)
    return visited
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}

print("BFS traversal starting from vertex A:")
bfs(graph, 'A')

BFS traversal starting from vertex A:


['A', 'B', 'C', 'D', 'E', 'F']

In [ ]:
def dfs(graph, start, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    print(start)
    for neighbor in graph[start]:
        if neighbor not in visited:
            dfs(graph, neighbor, visited)

graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}

print("DFS traversal starting from vertex A:")
dfs(graph, 'A')


DFS traversal starting from vertex A:
A
B
D
E
F
C


In [ ]:

import heapq

def a_star(graph, start, goal, heuristic):
    open_set = [(0, start)]
    came_from = {}
    cost_so_far = {start: 0}

    while open_set:
        _, current = heapq.heappop(open_set)
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        for neighbor, cost in graph.get(current, []):
            new_cost = cost_so_far[current] + cost
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost + heuristic(neighbor, goal)
                heapq.heappush(open_set, (priority, neighbor))
                came_from[neighbor] = current
    return None

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

graph = {
    (0, 0): [((1, 0), 1), ((0, 1), 1)],
    (1, 0): [((2, 0), 1), ((1, 1), 1), ((0, 0), 1)],
    (0, 1): [((1, 1), 1), ((0, 2), 1), ((0, 0), 1)],
    (2, 0): [((2, 1), 1), ((1, 0), 1)],
    (1, 1): [((2, 1), 1), ((1, 2), 1), ((1, 0), 1), ((0, 1), 1)],
    (0, 2): [((1, 2), 1), ((0, 1), 1)],
    (2, 1): [((2, 2), 1), ((1, 1), 1), ((2, 0), 1)],
    (1, 2): [((2, 2), 1), ((1, 1), 1), ((0, 2), 1)],
    (2, 2): [((2, 1), 1), ((1, 2), 1)],
}

start = (0, 0)
goal = (2, 2)
path = a_star(graph, start, goal, heuristic)
print(path)

[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]


In [ ]:
import heapq

# Goal state for the 8-puzzle problem
GOAL_STATE = [1, 2, 3, 4, 5, 6, 7, 8, 0]

# Directions in which the blank tile can move: up, down, left, right
MOVES = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # (row_change, col_change)

# Define a state as a list of tiles (0 represents the blank tile)
class PuzzleState:
    def __init__(self, state, parent=None, move=None, g=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.g = g  # Cost to reach this state (number of moves)
        self.h = self.calculate_heuristic()  # Heuristic cost (Manhattan distance)
        self.f = self.g + self.h  # f = g + h

    def calculate_heuristic(self):
        """Calculate the Manhattan distance of the current state to the goal state."""
        h = 0
        for i in range(9):
            if self.state[i] != 0:
                goal_pos = GOAL_STATE.index(self.state[i])
                current_row, current_col = divmod(i, 3)
                goal_row, goal_col = divmod(goal_pos, 3)
                h += abs(current_row - goal_row) + abs(current_col - goal_col)
        return h

    def get_possible_moves(self):
        """Get all possible valid moves (tile swaps) from the current state."""
        blank_pos = self.state.index(0)
        row, col = divmod(blank_pos, 3)
        moves = []

        for move in MOVES:
            new_row, new_col = row + move[0], col + move[1]
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_blank_pos = new_row * 3 + new_col
                new_state = self.state[:]
                new_state[blank_pos], new_state[new_blank_pos] = new_state[new_blank_pos], new_state[blank_pos]
                moves.append((new_state, (new_row, new_col)))

        return moves

    def __lt__(self, other):
        """Compare the states based on the f value (for priority queue)."""
        return self.f < other.f

    def __str__(self):
        """Convert the state to a string representation for easier printing."""
        return f"{self.state[0:3]}\n{self.state[3:6]}\n{self.state[6:9]}"

# A* Search Algorithm for solving the 8-puzzle
def a_star_search(start_state):
    start_node = PuzzleState(start_state)
    open_list = []
    heapq.heappush(open_list, start_node)
    closed_list = set()  # To avoid revisiting states

    while open_list:
        # Pop the state with the lowest f value
        current_node = heapq.heappop(open_list)

        # Check if we reached the goal state
        if current_node.state == GOAL_STATE:
            path = []
            while current_node:
                path.append(current_node.state)
                current_node = current_node.parent
            path.reverse()
            return path  # Return the solution path

        closed_list.add(tuple(current_node.state))

        # Get possible moves and explore them
        for new_state, move in current_node.get_possible_moves():
            if tuple(new_state) not in closed_list:
                new_node = PuzzleState(new_state, parent=current_node, move=move, g=current_node.g + 1)
                heapq.heappush(open_list, new_node)

    return None  # If no solution exists

def print_solution(path):
    if path:
        for step in path:
            print(step[0:3])
            print(step[3:6])
            print(step[6:9])
            print()
    else:
        print("No solution found!")

def main():
    # Initial state of the 8-puzzle (input puzzle)
    start_state = [1, 2, 3, 4, 0, 5, 7, 8, 6]

    # Perform A* search
    path = a_star_search(start_state)

    # Print the solution path (if exists)
    print_solution(path)

if __name__ == "__main__":
    main()

[1, 2, 3]
[4, 0, 5]
[7, 8, 6]

[1, 2, 3]
[4, 5, 0]
[7, 8, 6]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



In [8]:
import math

# Define the Minimax function
def minimax(node, depth, is_maximizing, values):
    # Base case: terminal node (leaf node)
    if depth == 3:  # Assuming leaf nodes are at depth 3
        return values[node]

    if is_maximizing:
        best_value = -math.inf
        # Maximizer chooses the maximum value
        for i in range(2):  # Each node has two children
            val = minimax(2 * node + i, depth + 1, False, values)
            best_value = max(best_value, val)
        return best_value
    else:
        best_value = math.inf
        # Minimizer chooses the minimum value
        for i in range(2):  # Each node has two children
            val = minimax(2 * node + i, depth + 1, True, values)
            best_value = min(best_value, val)
        return best_value


# Values at the terminal nodes (leaf nodes)
# Values correspond to the nodes at depth 3 in the tree
#terminal_values = [-1, 4, 2, 6, -3, -5, 0, 7]
terminal_values = [2,3,5,9,0,1,7,5]

# Root node is node 0 at depth 0
root = 0

# Perform the minimax algorithm
result = minimax(root, 0, True, terminal_values)
print("The optimal value at the root node is:", result)

The optimal value at the root node is: 3


In [9]:
import math

# Define the Alpha-Beta Pruning function
def alpha_beta(node, depth, is_maximizing, alpha, beta, values):
    # Base case: terminal node (leaf node)
    if depth == 3:  # Assuming leaf nodes are at depth 3
        return values[node]

    if is_maximizing:
        best_value = -math.inf
        # Maximizer chooses the maximum value
        for i in range(2):  # Each node has two children
            val = alpha_beta(2 * node + i, depth + 1, False, alpha, beta, values)
            best_value = max(best_value, val)
            alpha = max(alpha, best_value)
            # Prune the remaining branches if alpha >= beta
            if alpha >= beta:
                break
        return best_value
    else:
        best_value = math.inf
        # Minimizer chooses the minimum value
        for i in range(2):  # Each node has two children
            val = alpha_beta(2 * node + i, depth + 1, True, alpha, beta, values)
            best_value = min(best_value, val)
            beta = min(beta, best_value)
            # Prune the remaining branches if alpha >= beta
            if alpha >= beta:
                break
        return best_value


# Values at the terminal nodes (leaf nodes)
# Values correspond to the nodes at depth 3 in the tree
terminal_values = [2, 3, 5, 9, 0, 1, 4, 8]

# Root node is node 0 at depth 0
root = 0

# Perform the Alpha-Beta Pruning
optimal_value = alpha_beta(root, 0, True, -math.inf, math.inf, terminal_values)
print("The optimal value at the root node is:", optimal_value)

The optimal value at the root node is: 3
